# STOCK CLOSING PRICE PREDICTION

In [1]:
#importing libraries 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
import pandas_datareader as pdr

key="30c0973b8a648106ae38faea9031b9a3924c7469"

In [2]:
#Getting Dataset
df=pdr.get_data_tiingo('DIS', api_key=key)

C:\Users\sneha\AppData\Local\Temp\ipykernel_31488\3742500115.py:2: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  df=pdr.get_data_tiingo('DIS', api_key=key)


In [3]:
df.to_csv('DIS.csv')

In [4]:
df=pd.read_csv('DIS.csv', date_parser=True)

In [5]:
df.head()

,symbol,date,close,high,low,open,volume,adjClose,adjHigh,adjLow,adjOpen,adjVolume,divCash,splitFactor
0,DIS,2018-11-27 00:00:00+00:00,113.90,114.100,112.41,113.61,7357599,111.646841,111.842885,110.186316,111.362578,7357599,0.0,1.0
1,DIS,2018-11-28 00:00:00+00:00,116.10,116.300,114.05,114.32,9039821,113.803321,113.999365,111.793874,112.058533,9039821,0.0,1.0
2,DIS,2018-11-29 00:00:00+00:00,116.61,117.140,115.21,115.75,10834825,114.303232,114.822748,112.930927,113.460245,10834825,0.0,1.0
3,DIS,2018-11-30 00:00:00+00:00,115.49,117.065,115.06,116.84,16857777,113.205388,114.749232,112.783894,114.528683,16857777,0.0,1.0
4,DIS,2018-12-03 00:00:00+00:00,115.74,116.920,114.91,116.65,9272917,113.450443,114.607100,112.636862,114.342441,9272917,0.0,1.0


In [6]:
df.tail()

,symbol,date,close,high,low,open,volume,adjClose,adjHigh,adjLow,adjOpen,adjVolume,divCash,splitFactor
1252,DIS,2023-11-17 00:00:00+00:00,94.15,95.3770,93.92,94.75,12201304,94.15,95.3770,93.92,94.75,12201304,0.0,1.0
1253,DIS,2023-11-20 00:00:00+00:00,95.03,95.5850,93.87,93.98,14114766,95.03,95.5850,93.87,93.98,14114766,0.0,1.0
1254,DIS,2023-11-21 00:00:00+00:00,94.45,94.9600,93.96,94.73,7654843,94.45,94.9600,93.96,94.73,7654843,0.0,1.0
1255,DIS,2023-11-22 00:00:00+00:00,95.07,95.7061,94.76,94.91,7330932,95.07,95.7061,94.76,94.91,7330932,0.0,1.0
1256,DIS,2023-11-24 00:00:00+00:00,96.06,96.5100,94.90,94.90,5440468,96.06,96.5100,94.90,94.90,5440468,0.0,1.0


## Data Pre-processing

In [7]:
df['date'] = pd.to_datetime(df['date'])
# df['date'] = df['date'].dt.date
df

,symbol,date,close,high,low,open,volume,adjClose,adjHigh,adjLow,adjOpen,adjVolume,divCash,splitFactor
0,DIS,2018-11-27 00:00:00+00:00,113.90,114.1000,112.41,113.61,7357599,111.646841,111.842885,110.186316,111.362578,7357599,0.0,1.0
1,DIS,2018-11-28 00:00:00+00:00,116.10,116.3000,114.05,114.32,9039821,113.803321,113.999365,111.793874,112.058533,9039821,0.0,1.0
2,DIS,2018-11-29 00:00:00+00:00,116.61,117.1400,115.21,115.75,10834825,114.303232,114.822748,112.930927,113.460245,10834825,0.0,1.0
3,DIS,2018-11-30 00:00:00+00:00,115.49,117.0650,115.06,116.84,16857777,113.205388,114.749232,112.783894,114.528683,16857777,0.0,1.0
4,DIS,2018-12-03 00:00:00+00:00,115.74,116.9200,114.91,116.65,9272917,113.450443,114.607100,112.636862,114.342441,9272917,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1252,DIS,2023-11-17 00:00:00+00:00,94.15,95.3770,93.92,94.75,12201304,94.150000,95.377000,93.920000,94.750000,12201304,0.0,1.0
1253,DIS,2023-11-20 00:00:00+00:00,95.03,95.5850,93.87,93.98,14114766,95.030000,95.585000,93.870000,93.980000,14114766,0.0,1.0
1254,DIS,2023-11-21 00:00:00+00:00,94.45,94.9600,93.96,94.73,7654843,94.450000,94.960000,93.960000,94.730000,7654843,0.0,1.0
1255,DIS,2023-11-22 00:00:00+00:00,95.07,95.7061,94.76,94.91,7330932,95.070000,95.706100,94.760000,94.910000,7330932,0.0,1.0


In [8]:
# Splitting train and test set
df_training=df[df['date']<='2022-01-01'].copy()
df_test=df[df['date']>'2022-01-01'].copy()
df_test, df_training

(     symbol                      date   close      high       low    open  \
 780     DIS 2022-01-03 00:00:00+00:00  156.76  157.5600  155.3600  155.83   
 781     DIS 2022-01-04 00:00:00+00:00  155.73  160.3200  155.5500  158.59   
 782     DIS 2022-01-05 00:00:00+00:00  155.19  159.3800  155.1000  156.52   
 783     DIS 2022-01-06 00:00:00+00:00  156.90  157.7700  153.6791  156.24   
 784     DIS 2022-01-07 00:00:00+00:00  157.83  159.3000  156.2900  156.90   
 ...     ...                       ...     ...       ...       ...     ...   
 1252    DIS 2023-11-17 00:00:00+00:00   94.15   95.3770   93.9200   94.75   
 1253    DIS 2023-11-20 00:00:00+00:00   95.03   95.5850   93.8700   93.98   
 1254    DIS 2023-11-21 00:00:00+00:00   94.45   94.9600   93.9600   94.73   
 1255    DIS 2023-11-22 00:00:00+00:00   95.07   95.7061   94.7600   94.91   
 1256    DIS 2023-11-24 00:00:00+00:00   96.06   96.5100   94.9000   94.90   
 
         volume  adjClose   adjHigh    adjLow  adjOpen  adjVol

In [9]:
training_df=df_training.drop(['symbol','date','adjClose','adjHigh','adjLow','adjOpen','adjVolume','divCash','splitFactor'], axis=1)
training_df

,close,high,low,open,volume
0,113.90,114.100,112.410,113.61,7357599
1,116.10,116.300,114.050,114.32,9039821
2,116.61,117.140,115.210,115.75,10834825
3,115.49,117.065,115.060,116.84,16857777
4,115.74,116.920,114.910,116.65,9272917
...,...,...,...,...,...
775,152.80,154.020,151.400,152.88,7765470
776,155.20,156.500,152.410,152.62,12198679
777,154.87,155.920,154.180,154.56,8737004
778,155.93,157.070,155.515,155.71,7228438


## Feature Scaling

In [10]:
# scaler=MinMaxScaler(feature_range=(0,1))
# training_df=scaler.fit_transform(training_df)
scaler = MinMaxScaler(feature_range=(0, 1))
training_df_scaled = scaler.fit_transform(training_df)
training_df

array([[0.24227292, 0.23172628, 0.28659847, 0.25169627, 0.05474849],
       [0.26121395, 0.2507344 , 0.3006963 , 0.2578331 , 0.07461184],
       [0.26560482, 0.25799205, 0.31066793, 0.27019318, 0.09580689],
       ...,
       [0.59500646, 0.5930534 , 0.6456632 , 0.60564415, 0.07103623],
       [0.60413259, 0.60298946, 0.65713917, 0.61558408, 0.05322338],
       [0.59517865, 0.59866943, 0.65047709, 0.61696703, 0.04096006]])

## Converting into Dataset Matrix (taking previous 60 values)

In [11]:
X_train=[]
y_train=[]

In [12]:
training_df.shape

(780, 5)

In [13]:
# ltrain=training_df.shape[0]

# for i in range(60, ltrain):
#     X_train.append(training_df[i-60:i])
#     y_train.append(training_df[i,0])

for i in range(60, len(training_df_scaled)):
    X_train.append(training_df_scaled[i - 60:i, 1:])  # Adjusted to use columns 1 to end
    y_train.append(training_df_scaled[i, 0])

In [14]:
X_train, y_train=np.array(X_train), np.array(y_train)
X_train.shape, y_train.shape

((720, 60, 5), (720,))

## Building LSTM 

In [15]:
regressior=Sequential()
regressior.add(LSTM(units=60, activation='relu', return_sequences=True, input_shape=(X_train.shape[1],5)))
regressior.add(Dropout(0.2))

regressior.add(LSTM(units=60, activation='relu', return_sequences=True))
regressior.add(Dropout(0.2))

regressior.add(LSTM(units=80, activation='relu', return_sequences=True))
regressior.add(Dropout(0.2))

regressior.add(LSTM(units=120, activation='relu'))
regressior.add(Dropout(0.2))

regressior.add(Dense(units=1))

regressior.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 60, 60)            15840     
                                                                 
 dropout (Dropout)           (None, 60, 60)            0         
                                                                 
 lstm_1 (LSTM)               (None, 60, 60)            29040     
                                                                 
 dropout_1 (Dropout)         (None, 60, 60)            0         
                                                                 
 lstm_2 (LSTM)               (None, 60, 80)            45120     
                                                                 
 dropout_2 (Dropout)         (None, 60, 80)            0         
                                                                 
 lstm_3 (LSTM)               (None, 120)               9

In [16]:
regressior.compile(optimizer='adam', loss='mean_squared_error')
regressior.fit(X_train, y_train, epochs=50, batch_size=64)

Epoch 1/50
12/12 [==============================] - 22s 332ms/step - loss: 0.1727
Epoch 2/50
12/12 [==============================] - 5s 356ms/step - loss: 0.0303
Epoch 3/50
12/12 [==============================] - 4s 312ms/step - loss: 0.0162
Epoch 4/50
12/12 [==============================] - 4s 314ms/step - loss: 0.0108
Epoch 5/50
12/12 [==============================] - 4s 316ms/step - loss: 0.0100
Epoch 6/50
12/12 [==============================] - 4s 330ms/step - loss: 0.0106
Epoch 7/50
12/12 [==============================] - 4s 317ms/step - loss: 0.0098
Epoch 8/50
12/12 [==============================] - 4s 309ms/step - loss: 0.0085
Epoch 9/50
12/12 [==============================] - 4s 300ms/step - loss: 0.0086
Epoch 10/50
12/12 [==============================] - 4s 305ms/step - loss: 0.0077
Epoch 11/50
12/12 [==============================] - 4s 306ms/step - loss: 0.0083
Epoch 12/50
12/12 [==============================] - 4s 300ms/step - loss: 0.0073
Epoch 13/50
12/12 [=====

## Preparing Test Dataset

In [17]:
df_test.head()

,symbol,date,close,high,low,open,volume,adjClose,adjHigh,adjLow,adjOpen,adjVolume,divCash,splitFactor
780,DIS,2022-01-03 00:00:00+00:00,156.76,157.56,155.3600,155.83,10228257,156.76,157.56,155.3600,155.83,10228257,0.0,1.0
781,DIS,2022-01-04 00:00:00+00:00,155.73,160.32,155.5500,158.59,16582041,155.73,160.32,155.5500,158.59,16582041,0.0,1.0
782,DIS,2022-01-05 00:00:00+00:00,155.19,159.38,155.1000,156.52,12272097,155.19,159.38,155.1000,156.52,12272097,0.0,1.0
783,DIS,2022-01-06 00:00:00+00:00,156.90,157.77,153.6791,156.24,11095326,156.90,157.77,153.6791,156.24,11095326,0.0,1.0
784,DIS,2022-01-07 00:00:00+00:00,157.83,159.30,156.2900,156.90,9554622,157.83,159.30,156.2900,156.90,9554622,0.0,1.0


In [18]:
df_training.tail()

,symbol,date,close,high,low,open,volume,adjClose,adjHigh,adjLow,adjOpen,adjVolume,divCash,splitFactor
775,DIS,2021-12-27 00:00:00+00:00,152.80,154.02,151.400,152.88,7765470,152.80,154.02,151.400,152.88,7765470,0.0,1.0
776,DIS,2021-12-28 00:00:00+00:00,155.20,156.50,152.410,152.62,12198679,155.20,156.50,152.410,152.62,12198679,0.0,1.0
777,DIS,2021-12-29 00:00:00+00:00,154.87,155.92,154.180,154.56,8737004,154.87,155.92,154.180,154.56,8737004,0.0,1.0
778,DIS,2021-12-30 00:00:00+00:00,155.93,157.07,155.515,155.71,7228438,155.93,157.07,155.515,155.71,7228438,0.0,1.0
779,DIS,2021-12-31 00:00:00+00:00,154.89,156.57,154.740,155.87,6189861,154.89,156.57,154.740,155.87,6189861,0.0,1.0


In [19]:
past_60_days=df_training.tail(60)
# past_60_days
df1=past_60_days.append(df_test) #past 60 days taken for the first value of test data
df1=df1.drop(['symbol','date','adjClose','adjHigh','adjLow','adjOpen','adjVolume','divCash','splitFactor'], axis=1)
df1

C:\Users\sneha\AppData\Local\Temp\ipykernel_31488\1121012054.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df1=past_60_days.append(df_test) #past 60 days taken for the first value of test data


,close,high,low,open,volume
720,177.71,179.6300,177.3300,177.70,7779997
721,176.74,178.6000,176.4029,177.93,4132182
722,173.52,176.5200,173.4800,176.02,8046270
723,173.13,174.4900,172.5450,173.62,5188380
724,172.96,173.7000,170.9400,173.63,8918152
...,...,...,...,...,...
1252,94.15,95.3770,93.9200,94.75,12201304
1253,95.03,95.5850,93.8700,93.98,14114766
1254,94.45,94.9600,93.9600,94.73,7654843
1255,95.07,95.7061,94.7600,94.91,7330932


In [20]:
scaler=MinMaxScaler(feature_range=(0,1))
df1=scaler.fit_transform(df1)
df1

# inputs=scaler.transform(df1)
# inputs

array([[1.        , 1.        , 1.        , 0.99005924, 0.05515183],
       [0.99014127, 0.98968308, 0.99059728, 0.99236871, 0.        ],
       [0.95741437, 0.96884891, 0.96095299, 0.97319008, 0.05917765],
       ...,
       [0.15377579, 0.15190913, 0.15445476, 0.15694347, 0.05325961],
       [0.16007724, 0.15938239, 0.16256842, 0.15875088, 0.04836235],
       [0.17013924, 0.16743459, 0.16398831, 0.15865047, 0.01978016]])

In [21]:
df1.shape

(537, 5)

In [22]:
X_test=[]
y_test=[]

In [23]:
ltest=df1.shape[0]
for i in range(60,ltest):
    X_test.append(df1[i-60:i])
    y_test.append(df1[i,0])

In [24]:
X_test, y_test=np.array(X_test), np.array(y_test)
X_test.shape, y_test.shape

((477, 60, 5), (477,))

## Predicting the stock closing price

In [25]:
y_pred=regressior.predict(X_test)

15/15 [==============================] - 4s 87ms/step


In [26]:
# y_pred=scaler.inverse_transform(y_pred)

In [27]:
import math
from sklearn.metrics import mean_squared_error

math.sqrt(mean_squared_error(y_test, y_pred))

0.06907895826095639

In [28]:
y_test

array([7.87071857e-01, 7.76603313e-01, 7.71114951e-01, 7.88494766e-01,
       7.97946946e-01, 7.85445675e-01, 7.98556764e-01, 7.97642037e-01,
       7.73655859e-01, 7.38083139e-01, 7.41437138e-01, 7.19483687e-01,
       6.94176237e-01, 5.90100620e-01, 5.90913711e-01, 5.81258258e-01,
       5.51682082e-01, 5.70179896e-01, 6.02805163e-01, 6.46915337e-01,
       6.62364061e-01, 6.43358065e-01, 6.17034251e-01, 6.37259884e-01,
       6.42240065e-01, 6.41935156e-01, 6.90212420e-01, 7.40319138e-01,
       7.12978961e-01, 7.27004777e-01, 7.66338042e-01, 7.82904767e-01,
       7.48348409e-01, 7.32188231e-01, 6.98953146e-01, 6.76389877e-01,
       7.12267507e-01, 7.13588779e-01, 7.02713690e-01, 6.74662059e-01,
       6.91330420e-01, 6.73340787e-01, 6.24047159e-01, 5.50665718e-01,
       5.32879358e-01, 5.52190263e-01, 5.52088627e-01, 5.32879358e-01,
       5.05234272e-01, 5.57780262e-01, 5.97824982e-01, 6.11342616e-01,
       6.19778433e-01, 6.02703527e-01, 6.17847342e-01, 5.92743165e-01,
      

## Visualization the results

In [29]:
import plotly.graph_objs as go

# Create a trace for actual values
trace_actual = go.Scatter(x=df_test['date'], y=y_test, mode='lines', name='Actual', line=dict(color='red'))

# Create a trace for predicted values
trace_pred = go.Scatter(x=df_test['date'], y=y_pred.flatten(), mode='lines', name='Predicted', line=dict(color='green'))

# Create a trace for markers with reduced size
trace_markers1 = go.Scatter(
    x=df_test['date'],
    y=y_test.flatten(),
    mode='markers',
    name='',
    showlegend=False,
    marker=dict(color='red', size=3),  # Adjust the size here
)

trace_markers2 = go.Scatter(
    x=df_test['date'],
    y=y_pred.flatten(),
    mode='markers',
    name='',
    showlegend=False,
    marker=dict(color='green', size=3),  # Adjust the size here
)

# Create the Plotly figure
fig = go.Figure(data=[trace_actual, trace_pred, trace_markers1, trace_markers2])

# Customize the layout
fig.update_layout(
    title='Stock Price Prediction',
    xaxis_title='Time',
    yaxis_title='Stock Price',
     width=800,
    height=600
)

# Show the Plotly figure
fig.show()


In [44]:
df_past_60_days_for_future = df_test.tail(60).copy()
df_past_60_days_for_future=df_past_60_days_for_future.drop(['symbol','date','adjClose','adjHigh','adjLow','adjOpen','adjVolume','divCash','splitFactor'], axis=1)
df_past_60_days_for_future

X_input=list(df_past_60_days_for_future)
X_input = df_past_60_days_for_future.iloc[:, 0].tolist()
X_input

# demonstrate prediction for next 10 days
from numpy import array
import pandas as pd 

lst_output=[]
n_steps=60
i=0

while(i<10):
        
        if(len(temp_input)>60):
            #print(temp_input)
            X_input=np.array(temp_input[1:])
            print("{} day input {}".format(i,X_input))
            X_input=X_input.reshape(1,-1)
            X_input = X_input.reshape((1, n_steps, 1))
            #print(x_input)
            yhat = regressior.predict(X_input, verbose=0)
            print("{} day output {}".format(i,yhat))
            temp_input.extend(yhat[0].tolist())
            temp_input=temp_input[1:]
            #print(temp_input)
            lst_output.extend(yhat.tolist())
            i=i+1
        else:
            X_input = X_input.reshape((1, n_steps,1))
            yhat = regressior.predict(X_input, verbose=0)
            print(yhat[0])
            temp_input.extend(yhat[0].tolist())
            print(len(temp_input))
            lst_output.extend(yhat.tolist())
            i=i+1

print(lst_output)

# day_new=np.arange(1,61)
# day_pred=np.arange(61,71)
# plt.plot(day_new,df_train['Close'],'b',label='Actual')
# plt.plot(day_pred,lst_output,'r',label='Predicted')
# plt.legend()
# plt.show()




AttributeError: 'list' object has no attribute 'reshape'

In [31]:
# X_train.shape[1]
print(len(temp_input))


60


In [41]:
# # ...

# # demonstrate prediction for next 10 days
# lst_output = []
# n_steps = 60
# i = 0

# while i < 10:
#     if len(temp_input) >= 60:
#         df_past_60_days_for_future = np.array(temp_input[-60:]).reshape((1, 60, 5))
#     else:
#         df_past_60_days_for_future = np.array(temp_input + [0] * (60 - len(temp_input))).reshape((1, 60, 5))

#     yhat = regressior.predict(df_past_60_days_for_future, verbose=0)[:, 0]    
#     temp_input.extend(yhat[0].tolist())
#     lst_output.extend(yhat.tolist())
#     i = i + 1

# print(lst_output)


# ...


ValueError: cannot reshape array of size 60 into shape (1,60,5)

In [ ]:
df_past_60_days_for_future

,close,high,low,open,volume
1198,83.68,84.6100,83.6400,84.460,10447960
1199,81.64,82.2300,80.5300,82.105,27701889
1200,81.19,82.3850,81.1750,81.420,12741682
1201,80.98,82.0900,80.5300,80.790,15518544
1202,80.57,80.8500,79.7500,80.820,14767972
1203,81.58,81.6700,80.1700,80.540,16233621
1204,82.52,83.8500,81.1700,81.940,26579061
1205,83.68,84.6700,82.5700,83.030,16594608
1206,83.48,84.0450,83.1200,83.350,9772692
1207,84.48,84.7400,83.0400,84.000,22371918
